# Fondos de inversión Bancolombia

Este cuaderno contiene los pasos de un procedimiento en el cual se extraen datos de la página de [fondos de inversión](https://www.bancolombia.com/personas/productos-servicios/inversiones/fondos-inversion-colectiva/aplicacion-fondos?_ga=2.238516480.1238354001.1676407609-1550703317.1664458021) de Bancolombia.


Importamos las librerías necesarias para la ejecución de este cuaderno.

In [4]:
import pandas as pd
import requests
import io
from urllib.parse import urlencode

import sys
sys.path.append('c:\\Users\\mauricio.cano\\Desktop\\Proyectos\\AZ900-Azure-Fundamentals\\')

from analisisrentabilidades.limpieza import limpieza_datos_retabilidades

Definimos la URL principal de donde tomaremos los nombres de los fondos de inversión que necesitamos.


In [5]:
url = "https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/consultarListaFondos"
url_base =  "https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/"

Le pedimos con al código obtener la información de la URL anteriormente definida con el comando get( ). Posteriormente se pide la extracción de la información como salida con .content. 

In [6]:
response= requests.get(url)
response.status_code
response.content

b'[{"nit":"800241308","nombre":"Renta Balanceado"},{"nit":"800227622","nombre":"Plan Semilla"},{"nit":"900000528","nombre":"Renta Acciones"},{"nit":"900000531","nombre":"Renta Fija Plazo"},{"nit":"800244627","nombre":"Fidurenta"},{"nit":"900237003","nombre":"Fiduexcedentes"},{"nit":"800180687","nombre":"Fiducuenta"},{"nit":"9012636163","nombre":"FICI Sura AM Rentas Inmobiliarias"},{"nit":"901536211","nombre":"Renta Alternativo Global"},{"nit":"901446225-3","nombre":"Cerrado Renta Fija"},{"nit":"901573146-3","nombre":"Cerrado Renta Fija II"},{"nit":"901574370-1","nombre":"Cerrado Renta Fija III"},{"nit":"901574835-4","nombre":"Fondo Cerrado Renta Fija IV"},{"nit":"9004381462","nombre":"Renta Liquidez"},{"nit":"9004724683","nombre":"Renta Variable Colombia"},{"nit":"9006349247","nombre":"Renta Sostenible Global"},{"nit":"9008966743","nombre":"Renta Fija Plus"},{"nit":"9009585538","nombre":"Renta Alta Convicci\xc3\xb3n"},{"nit":"9013917439","nombre":"Renta Futuro"}]'

Utilizamos .json( ) para poder convertir nuestro objeto en una lista y la definimos 'fondo'.

In [10]:
fondo=response.json()
fondo

[{'nit': '800241308', 'nombre': 'Renta Balanceado'},
 {'nit': '800227622', 'nombre': 'Plan Semilla'},
 {'nit': '900000528', 'nombre': 'Renta Acciones'},
 {'nit': '900000531', 'nombre': 'Renta Fija Plazo'},
 {'nit': '800244627', 'nombre': 'Fidurenta'},
 {'nit': '900237003', 'nombre': 'Fiduexcedentes'},
 {'nit': '800180687', 'nombre': 'Fiducuenta'},
 {'nit': '9012636163', 'nombre': 'FICI Sura AM Rentas Inmobiliarias'},
 {'nit': '901536211', 'nombre': 'Renta Alternativo Global'},
 {'nit': '901446225-3', 'nombre': 'Cerrado Renta Fija'},
 {'nit': '901573146-3', 'nombre': 'Cerrado Renta Fija II'},
 {'nit': '901574370-1', 'nombre': 'Cerrado Renta Fija III'},
 {'nit': '901574835-4', 'nombre': 'Fondo Cerrado Renta Fija IV'},
 {'nit': '9004381462', 'nombre': 'Renta Liquidez'},
 {'nit': '9004724683', 'nombre': 'Renta Variable Colombia'},
 {'nit': '9006349247', 'nombre': 'Renta Sostenible Global'},
 {'nit': '9008966743', 'nombre': 'Renta Fija Plus'},
 {'nit': '9009585538', 'nombre': 'Renta Alta Co

Convertimos la lista anterior en un Dataframe.

In [9]:
Fondodf= pd.DataFrame(fondo, columns= ['nit', 'nombre'])
Fondodf

NameError: name 'fondo' is not defined

In [ ]:
fondo[0]

Extraemos los códigos NIT de cada fondo de inversión.

In [11]:
codigo = [fondo[i]["nit"] for i in range(len(fondo))]
print(codigo)

['800241308', '800227622', '900000528', '900000531', '800244627', '900237003', '800180687', '9012636163', '901536211', '901446225-3', '901573146-3', '901574370-1', '901574835-4', '9004381462', '9004724683', '9006349247', '9008966743', '9009585538', '9013917439']


Una forma alternativa es extraer los NIT del Dataframe.

In [ ]:
Fondodf['nit'].tolist()

Definimos la URL que nos proporciona la información completa de rentabilidad y la juntamos con los códigos que extrajimos anteriormente. 

In [12]:
consultas= [url_base + codigo[i] for i in range(len(codigo))]
consultas

['https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/800241308',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/800227622',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/900000528',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/900000531',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/800244627',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/900237003',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/800180687',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest/servicio/buscarInformacionFondo/9012636163',
 'https://fiduciaria.grupobancolombia.com/consultarFondosInversion/rest

Solicitamos a las URL anteriormente generadas, extraer la información de rentabilidad de cada fondo.

In [13]:
infofondos=[requests.get(consultas[i]).json() for i in range(len(consultas))]
infofondos

[{'nit': '800241308',
  'nombre': 'Renta Balanceado',
  'calificacion': 'No Calificado',
  'plazo': 'A la vista',
  'valorDeUnidad': '15.488,90',
  'valorEnPesos': '$83,337,128,227.90',
  'rentabilidad': {'dias': {'semanal': '27,65%',
    'mensual': '19,76%',
    'semestral': '5,47%'},
   'anios': {'anioCorrido': '37,12%',
    'ultimoAnio': '-1,43%',
    'ultimos2Anios': '0,08%',
    'ultimos3Anios': '1,57%'}},
  'fechaCierre': '20230216',
  'sociedadAdministradora': 'Fiduciaria Bancolombia S.A.'},
 {'nit': '800227622',
  'nombre': 'Plan Semilla',
  'calificacion': '(3/AAA)',
  'plazo': '1 aÂ¦o',
  'valorDeUnidad': '24.233,62',
  'valorEnPesos': '$632,250,015,609.81',
  'rentabilidad': {'dias': {'semanal': '34,83%',
    'mensual': '34,59%',
    'semestral': '14,90%'},
   'anios': {'anioCorrido': '33,47%',
    'ultimoAnio': '5,50%',
    'ultimos2Anios': '1,20%',
    'ultimos3Anios': '3,26%'}},
  'fechaCierre': '20230216',
  'sociedadAdministradora': 'Fiduciaria Bancolombia S.A.'},
 {'ni

Generamos un dataframe con la información de las claves y valores en el diccionario, omitiendo en principio los subdiccionarios de **rentabilidad** en _días_ y _años_.

In [14]:
Totaldf = limpieza_datos_retabilidades(infofondos)

c:\Users\mauricio.cano\Desktop\Proyectos\AZ900-Azure-Fundamentals\analisisrentabilidades\limpieza.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  anual = Totaldf["rentabilidad"].apply(pd.Series).iloc[:,:4].copy()
c:\Users\mauricio.cano\Desktop\Proyectos\AZ900-Azure-Fundamentals\analisisrentabilidades\limpieza.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  anual = Totaldf["rentabilidad"].apply(pd.Series).iloc[:,:4].copy()
c:\Users\mauricio.cano\Desktop\Proyectos\AZ900-Azure-Fundamentals\analisisrentabilidades\limpieza.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  diarios = Totaldf["rentabilidad"].apply(pd.Series).iloc[:,4:].

In [15]:
Totaldf

,nit,nombre,calificacion,plazo,valorDeUnidad,valorEnPesos,fechaCierre,sociedadAdministradora,anioCorrido,ultimoAnio,ultimos2Anios,ultimos3Anios,semanal,mensual,semestral
0,800241308,Renta Balanceado,No Calificado,A la vista,"15.488,90","$83,337,128,227.90",20230216,Fiduciaria Bancolombia S.A.,"37,12%","-1,43%","0,08%","1,57%","27,65%","19,76%","5,47%"
1,800227622,Plan Semilla,(3/AAA),1 aÂ¦o,"24.233,62","$632,250,015,609.81",20230216,Fiduciaria Bancolombia S.A.,"33,47%","5,50%","1,20%","3,26%","34,83%","34,59%","14,90%"
2,900000528,Renta Acciones,No Calificado,A la Vista,"51.101,62","$34,105,637,256.49",20230216,Fiduciaria Bancolombia S.A.,"-42,01%","-17,58%","-5,81%","-8,18%","-56,59%","-72,79%","-15,33%"
3,900000531,Renta Fija Plazo,S5/AAAf(col),N/A,"29.526,15","$207,266,688,604.50",20230216,Fiduciaria Bancolombia S.A.,"39,82%","-0,12%","-2,95%","1,16%","-0,67%","26,00%","14,84%"
4,800244627,Fidurenta,S3/AAAf(col),30 a 59 dÃ­as; 60 a 89 dÃ­as; 90 a 179dÃ­as; 1...,"37.873,46","$459,354,601,862.60",20230216,Fiduciaria Bancolombia S.A.,"26,71%","7,40%","3,42%","4,26%","44,94%","32,02%","15,73%"
5,900237003,Fiduexcedentes,(1/AAA),A la vista,"17.951,35","$1,268,728,624,214.63",20230216,Fiduciaria Bancolombia S.A.,"19,87%","8,55%","4,60%","4,17%","28,97%","22,17%","13,26%"
6,800180687,Fiducuenta,(1/AAA),A la Vista,"33.818,92","$12,419,825,949,596.00",20230216,Fiduciaria Bancolombia S.A.,"19,67%","8,73%","4,64%","4,09%","28,43%","22,13%","13,23%"
7,9012636163,FICI Sura AM Rentas Inmobiliarias,No Calificado,N/A,"12.726,75","$452,892,882,129.42",20230216,Fiduciaria Bancolombia S.A.,"2,15%","6,60%","5,41%","3,02%","11,83%","0,99%","7,62%"
8,901536211,Renta Alternativo Global,No Calificado,Result=N/A,"9.861,74","$254,780,828,144.96",20230216,Fiduciaria Bancolombia S.A.,"11,62%","-1,38%",N/A%,N/A%,"-28,29%","-8,23%","-2,70%"
9,901446225-3,Cerrado Renta Fija,No Calificado,Result=N/A,"10.597,55",$.05-,NaN,Fiduciaria Bancolombia S.A.,"29,98%",N/A%,N/A%,N/A%,"48,46%","38,01%","11,43%"


In [ ]:
Totaldf.head()

In [ ]:
Totaldf= pd.DataFrame((infofondos[i] for i in range(len(infofondos))), index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], columns= ['nit', 'nombre','calificacion', 'plazo', 'valorDeUnidad','valorEnPesos', 'fechaCierre', 'sociedadAdministradora'])
Totaldf

In [ ]:
extrentab = []
for infofondos for i in infofondos:
    extrentabilidad = infofondos[i]['rentabilidad']
    extrentab.append(extrentabilidad)

In [ ]:

primeros_elementos = [['rentabilidad']]
primeros_elementos

In [ ]:
rentdic0=intento['rentabilidad']
dias0=rentdic0['dias']
print(dias0)
anios0=rentdic0['anios']
print(anios0)


In [ ]:
dias0df= pd.DataFrame(dias0, index=[1], columns= ['semanal', 'mensual','semestral'])
dias0df


In [ ]:
anios0df= pd.DataFrame(anios0, index=[1], columns= ['anioCorrido', 'ultimoAnio','ultimos2Anios', 'ultimos3Anios'])
anios0df

In [ ]:
rentab0 = pd.concat([intentodf,dias0df, anios0df], axis=1)
rentab0

In [ ]:
type(rentabilidad[18])